# Backpropagation

Backpropogation algorithm is used to train a neural network by adjusting the weights and biases to minimize the cost function. It is an iterative process that involves forward propagation and backward propagation. 

## How does a Neural Network learn? 
The learning process is simply adjusting the weights and biases that's it! The Neural Netowork does this by a process called Backpropagation. The steps are as follows:
1. Randomly **initialise weights**
2. __Forward Pass__: Predict a value using an activation function. 
3. See how bad you're performing using loss function (compare predicted value with actual value). 
4. __Backward Pass__: Backpropagate the error. That is, tell your network that it's wrong, and also tell what direction it's supposed to go in order to reduce the error. This step updates the weights (here's where the network learns!)
5. Repeat steps 2 & 3 until the error is reasonably small or for a specified number of iterations. 

Step 3 is the most important step. We'll mathematically derive the equation for updating the values. 


## Random Initialization

Before starting forward propagation we need to initialize Theta parameters. We can not assign zero to all thetas since this would make our network useless because every neuron of the layer will learn the same as its siblings. In other word we need to **break the symmetry**. In order to do so we need to initialize thetas to some small random initial values:

![theta-init](../images/theta-init.svg)

## Forward (or Feedforward) Propagation

Forward propagation is an interactive process of calculating activations for each layer starting from the input layer and going to the output layer.

For the simple network mentioned in a previous section above we're able to calculate activations for second layer based on the input layer and our network parameters:

![a-1-2](../images/a-1-2.svg)

![a-2-2](../images/a-2-2.svg)

![a-3-2](../images/a-3-2.svg)

The output layer activation will be calculated based on the hidden layer activations:

![h-Theta-example](../images/h-Theta-example.svg)

Where _g()_ function may be a sigmoid:

![sigmoid](../images/sigmoid.svg)

![Sigmoid](https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg)

### Vectorized Implementation of Forward Propagation

Now let's convert previous calculations into more concise vectorized form.

![neuron x](../images/neuron-x.svg)

To simplify previous activation equations let's introduce a _z_ variable:

![z-1](../images/z-1.svg)

![z-2](../images/z-2.svg)

![z-3](../images/z-3.svg)

![z-matrix](../images/z-matrix.svg)

> Don't forget to add bias units (activations) before propagating to the next layer.
> ![a-bias](../images/a-bias.svg)

![z-3-vectorize](../images/z-3-vectorized.svg)

![h-Theta-vectorized](../images/h-Theta-vectorized.svg)

### Forward Propagation Example

Let's take the following network architecture with 4 layers (input layer, 2 hidden layers and output layer) as an example:

![multi-class-network](../images/multi-class-network.drawio.svg)

In this case the forward propagation steps would look like the following:

![forward-propagation-example](../images/forward-propagation-example.svg)


## Cost Function

Cost function is used to check how bad you're network is performing. One simple way to do that is subtract the predicted value and the actual value. For instance, if the actual value is 45 and your network is predicting 30, you can extrapolate you network is off by 15. 

In practise we don't use the simple subtraction, we instead **square**. Why do we square it? 

Well the square function has better mathematical properties (like it's a convex function and it's differentiable) which makes it easier for us to calculate the gradient.  

To calculate the loss function, we will perform derivation of the loss function with respect to the weights. 

$$ \tag 1
MSE=\sum_{i=0}^m\left(y^{\left(i\right)} - h_{\theta}\left(x^{\left(i\right)}\right)\right)^2
$$

Which is simply the sum of the squared difference between the obeserved value and predicted value. 

In the above equation
1. m = number of examples
2. $h(\theta)$ = activation function
3. $x^{(i)}$ = the ith sample in the dataset
4. $y^{(i)}$ = output of ith sample in the dataset

We'll be using sigmoid activation function which is simplest to illustrate while deriving Backpropagation. 

Derivation of the sigmoid function looks like this:

$$ \tag 2
\frac{\partial}{\partial x}\sigma\left(x\right)=\sigma\left(x\right)\cdot\left(1-\sigma\left(x\right)\right)
$$

We'll denote the above equation by $\sigma'\left(x\right)$


The cost function for the neuron network is quite similar to the logistic regression cost function.


![cost-function](../images/cost-function.svg)

![h-Theta](../images/h-Theta.svg)

![h-Theta-i](../images/h-Theta-i.svg)


## Backpropagation

### Gradient Computation

Backpropagation algorithm has the same purpose as gradient descent for linear or logistic regression - it corrects the values of thetas to minimize a cost function.

In other words we need to be able to calculate partial derivative of cost function for each theta.

![J-partial](../images/J-partial.svg)

![multi-class-network](../images/multi-class-network.drawio.svg)

Let's assume that:

![delta-j-l](../images/delta-j-l.svg) - "error" of node _j_ in layer _l_.

For each output unit (layer _L = 4_):

![delta-4](../images/delta-4.svg)

Or in vectorized form:

![delta-4-vectorized](../images/delta-4-vectorized.svg)

![delta-3-2](../images/delta-3-2.svg)

![sigmoid-gradient](../images/sigmoid-gradient.svg) - sigmoid gradient.

![sigmoid-gradient-2](../images/sigmoid-gradient-2.svg)

Now we may calculate the gradient step:

![J-partial-detailed](../images/J-partial-detailed.svg)

### Backpropagation Algorithm

For training set

![training-set](../images/training-set.svg)

We need to set:

![Delta](../images/Delta.svg)

![backpropagation](../images/backpropagation.svg)

## Implementation 

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from scipy.io import loadmat

data = loadmat("machine_learning_andrewng/ex4data1.mat")
print(data.keys())
print(weights.keys())

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])
dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])


In [8]:
X = data['X']
y = data['y']
#one-hot encoding the y values
y = pd.get_dummies(y.ravel()).values

In [9]:
import numpy as np
import pandas as pd

from scipy.optimize import minimize
from time import time

class NeuralNetwork:
    """Creates Neural Network for MNIST dataset."""
    def __init__(self, hidden_size=25, output_size=10, lambda_=0):
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.thetas = None
        self.lambda_ = lambda_
        self.X = None
        self.y = None
        self.input_size = 0

    @staticmethod
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def flatten(arr1, arr2):
        return np.r_[arr1.flatten(), arr2.flatten()]
    
    def set_params(self, *thetas):
        self.thetas = self.flatten(*thetas)

    def unflatten(self, arr):
        theta1 = arr[:self.hidden_size * (self.input_size + 1)]
        theta1 = theta1.reshape(self.hidden_size, self.input_size + 1)
        theta2 = arr[self.hidden_size * (self.input_size + 1):]
        theta2 = theta2.reshape(self.output_size, self.hidden_size + 1)
        return theta1, theta2

    def init_random_thetas(self, epsilon=0.12):
        theta1 = np.random.rand(self.hidden_size, self.input_size + 1) \
                 * 2 * epsilon - epsilon
        theta2 = np.random.rand(self.output_size, self.hidden_size + 1) \
                 * 2 * epsilon - epsilon
        return self.flatten(theta1, theta2)

    def sigmoid_prime(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    #loss function
    def cross_entropy(self, thetas=None):
        if thetas is None:
            theta1, theta2 = self.unflatten(self.thetas)
        else:
            theta1, theta2 = self.unflatten(thetas)
        m = self.X.shape[0]
        y_pred = self.forward_pass(thetas)
        positive_loss = np.sum(np.multiply(self.y, np.log(y_pred)).flatten())
        negative_loss = np.sum(np.multiply((1 - self.y), np.log(1 - y_pred))
                               .flatten())
        regularization = (self.lambda_ / (2 * m)) \
                         * (np.sum(theta1.flatten() ** 2)
                            + np.sum(theta2.flatten() ** 2))
        J = - (1 / m) * (positive_loss + negative_loss) + regularization
        return J

    def forward_pass(self, thetas=None, elaborate=False):
        if thetas is None:
            theta1, theta2 = self.unflatten(self.thetas)
        else:
            theta1, theta2 = self.unflatten(thetas)
        a1 = np.c_[np.ones(self.X.shape[0]), self.X]
        z2 = theta1.dot(a1.T)                   # 25x401 * 401x5000 = 25x5000
        a2 = self.sigmoid(z2.T)                 # 5000x25
        a2 = np.c_[np.ones(a2.shape[0]), a2]    # 5000x26
        z3 = theta2.dot(a2.T)                   # 10x26 * 26x5000 = 10x5000
        a3 = self.sigmoid(z3.T)                 # 5000x10
        if elaborate:
            return (a1, a2, a3), (z2, z3)
        return a3

    def backward_pass(self, thetas=None):
        if thetas is None:
            theta1, theta2 = self.unflatten(self.thetas)
        else:
            theta1, theta2 = self.unflatten(thetas)
        (a1, a2, y_pred), (z2, z3) = self.forward_pass(thetas, elaborate=True)
        delta3 = np.multiply((y_pred - self.y), self.sigmoid_prime(z3.T))
        theta2_grad = a2.T.dot(delta3)
        theta2_grad = theta2_grad.T
        # theta2_grad.shape is now same as theta2.shape
        delta2 = np.multiply(delta3.dot(theta2[:, 1:]), self.sigmoid_prime(z2.T))
        theta1_grad = a1.T.dot(delta2)
        theta1_grad = theta1_grad.T
        return self.flatten(theta1_grad, theta2_grad)

    def gradient_descent(self, X, y, n_epochs=1000, alpha=0.001):
        self.thetas = self.init_random_thetas()
        theta1, theta2 = self.unflatten(self.thetas)
        
        for i in range(1, n_epochs+1):
            cost = self.cross_entropy()
            print("\rIteration: {0} Cost: {1}".format(i, cost), end="")
            theta1_grad, theta2_grad = self.unflatten(self.backward_pass())
            theta1 = theta1 - alpha * theta1_grad
            theta2 = theta2 - alpha * theta2_grad
            self.thetas = self.flatten(theta1, theta2)
        print()
        
    def fmin_unc(self, X, y, **params):
        self.thetas = self.init_random_thetas()
        res = minimize(self.cross_entropy, self.thetas, jac=self.backward_pass,
                       method="tnc", options=params)
        print(res)

    def fit(self, X, y):
        if y.shape[1] != self.output_size:
            raise ValueError("Number of columns in y ({0}) are != to number "
                             "of output neurons ({1})"
                             .format(y.shape[1], self.output_size))
        self.X = X
        self.y = y
        self.input_size = X.shape[1]
    
    def train(self, method="gradient_descent", **params):
        start_time = time()
        if method == "gradient_descent":
            self.gradient_descent(X, y, **params)
        else:
            self.fmin_unc(X, y, **params)
        print("Training time: {0:.2f} secs".format(time() - start_time))


In [10]:
nn = NeuralNetwork(hidden_size=25, output_size=10)
nn.fit(X, y)
print("Training using Gradient Descent")
nn.train()

Training using Gradient Descent
Iteration: 1000 Cost: 0.4512534813094702
Training time: 133.12 secs


In [5]:
nn = NeuralNetwork(hidden_size=25, output_size=10)
nn.fit(X, y)
print("Training using Newton's Conjugate Gradient")
nn.train(method="newton", maxiter=1000)

Training using Newton's Conjugate Gradient


/var/folders/7m/04ssj6n96q984_6wsnr60dg00000gn/T/ipykernel_19899/3101508393.py:108: OptimizeWarning: Unknown solver options: maxiter
  res = minimize(self.cross_entropy, self.thetas, jac=self.backward_pass,
/var/folders/7m/04ssj6n96q984_6wsnr60dg00000gn/T/ipykernel_19899/3101508393.py:55: RuntimeWarning: divide by zero encountered in log
  negative_loss = np.sum(np.multiply((1 - self.y), np.log(1 - y_pred))
/var/folders/7m/04ssj6n96q984_6wsnr60dg00000gn/T/ipykernel_19899/3101508393.py:55: RuntimeWarning: invalid value encountered in multiply
  negative_loss = np.sum(np.multiply((1 - self.y), np.log(1 - y_pred))


 message: Linear search failed
 success: False
  status: 4
     fun: 1.1439424009572163
       x: [-1.856e-01  1.621e-02 ... -3.700e-01 -6.315e+00]
     nit: 25
     jac: [-2.211e-02  0.000e+00 ... -8.103e-02  2.362e-01]
    nfev: 542
Training time: 47.70 secs


In [6]:
nn = NeuralNetwork(hidden_size=25, output_size=10)
nn.fit(X, y)
nn.set_params(theta1_loaded, theta2_loaded)
nn.cross_entropy()

0.2876291651613189

## References

- [Machine Learning on Coursera](https://www.coursera.org/learn/machine-learning)
- [But what is a Neural Network? By 3Blue1Brown](https://www.youtube.com/watch?v=aircAruvnKk)
- [Neural Network on Wikipedia](https://en.wikipedia.org/wiki/Artificial_neural_network)
- [TensorFlow Neural Network Playground](https://playground.tensorflow.org/)
- [Deep Learning by Carnegie Mellon University](https://insights.sei.cmu.edu/sei_blog/2018/02/deep-learning-going-deeper-toward-meaningful-patterns-in-complex-data.html)
